# Business Problem section
## Background

According to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. More specifically, four overlooked cracks suggest that the London market may be in worse shape than many realize: hidden price falls, record-low sales, homebuilder exodus and tax hikes addressing overseas buyers of homes in England and Wales.

## Business Problem

In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

## Data section

Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.

## Methodology section
The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

### 1. Collect Inspection Data
After importing the necessary libraries, we download the data from the HM Land Registry website as follows:

In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

Before using data, we will have to explore and understand it.

### 2. Explore and Understand Data
We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows:

In [4]:
df_ppd.head()

,{79A74E21-C934-1289-E053-6B04A8C01627},177000,2018-09-21 00:00,LE4 6EE,S,N,F,201,Unnamed: 8,BELPER STREET,Unnamed: 10,LEICESTER,LEICESTER.1,LEICESTER.2,A,A.1
0,{79A74E21-C935-1289-E053-6B04A8C01627},90000,2018-10-01 00:00,LE18 2AE,F,N,L,27,NaN,ELIZABETH COURT,NaN,WIGSTON,OADBY AND WIGSTON,LEICESTERSHIRE,A,A
1,{79A74E21-C936-1289-E053-6B04A8C01627},375000,2018-10-04 00:00,LE11 3HG,D,N,F,6,NaN,GOLDFINCH CLOSE,NaN,LOUGHBOROUGH,CHARNWOOD,LEICESTERSHIRE,A,A
2,{79A74E21-C937-1289-E053-6B04A8C01627},142500,2018-10-08 00:00,LE3 6UY,S,N,F,19,NaN,PINEHURST CLOSE,NaN,LEICESTER,LEICESTER,LEICESTER,A,A
3,{79A74E21-C938-1289-E053-6B04A8C01627},157500,2018-10-22 00:00,LE13 0JH,S,N,F,103,NaN,WEST AVENUE,NaN,MELTON MOWBRAY,MELTON,LEICESTERSHIRE,A,A
4,{79A74E21-C939-1289-E053-6B04A8C01627},192500,2018-10-02 00:00,LE12 7UT,T,N,F,26,NaN,MELODY DRIVE,SILEBY,LOUGHBOROUGH,CHARNWOOD,LEICESTERSHIRE,A,A


In [5]:
df_ppd.shape

(1017356, 16)

Our dataset consists of over 700000 rows and 16 columns. We will now prepare and preprocess data accordingly.

### 3. Data preparation and preprocessing
At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps:

- Rename the column names
- Format the date column
- Sort data by date of sale
- Select data only for the city of London
- Make a list of street names in London
- Calculate the street-wise average price of the property
- Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names
- Join the data to find the coordinates of locations which fit into client's budget
- Plot recommended locations on London map along with current market prices

In [6]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [7]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [8]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [9]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [10]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [11]:
# Display the dataframe
df_affordable

,Street,Avg_Price
195,ALBION SQUARE,2.450000e+06
390,ANHALT ROAD,2.435000e+06
405,ANSDELL TERRACE,2.250000e+06
420,APPLEGARTH ROAD,2.400000e+06
698,AYLESTONE AVENUE,2.286667e+06
851,BARONSMEAD ROAD,2.375000e+06
975,BEAUCLERC ROAD,2.480000e+06
1096,BELVEDERE DRIVE,2.340000e+06
1209,BICKENHALL STREET,2.208500e+06
1247,BIRCHLANDS AVENUE,2.217000e+06


In [12]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [13]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 195
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 195, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 420
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 420, dtype: object
item.Street only: APPLEGARTH ROAD
index: 698
item: Street       AYLESTONE AVENUE
Avg_Price         2.28667e+06
Name: 698, dtype: object
item.Street only: AYLESTONE AVENUE
index: 851
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 851, dtype: object
item.Street only: BARONSMEAD ROAD
index: 975
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 975, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1096
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name

In [21]:
geolocator = Nominatim(timeout=20)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [22]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
df_affordable

,Street,Avg_Price,city_coord
195,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)"
390,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)"
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
420,APPLEGARTH ROAD,2.400000e+06,"(53.749244, -0.32678)"
698,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)"
851,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)"
975,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)"
1096,BELVEDERE DRIVE,2.340000e+06,"(38.201316, -84.623076)"
1209,BICKENHALL STREET,2.208500e+06,"(51.5211969, -0.1589341)"
1247,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)"


In [24]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [25]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
195,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)",-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)",51.480326,-0.166761
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
420,APPLEGARTH ROAD,2.400000e+06,"(53.749244, -0.32678)",53.749244,-0.326780
698,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)",51.540916,-0.217874
851,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
975,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)",51.499577,-0.229033
1096,BELVEDERE DRIVE,2.340000e+06,"(38.201316, -84.623076)",38.201316,-84.623076
1209,BICKENHALL STREET,2.208500e+06,"(51.5211969, -0.1589341)",51.521197,-0.158934
1247,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)",51.448394,-0.160468


In [26]:
df = df_affordable.drop(columns=['city_coord'])

In [27]:
df

,Street,Avg_Price,Latitude,Longitude
195,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
420,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780
698,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874
851,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
975,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033
1096,BELVEDERE DRIVE,2.340000e+06,38.201316,-84.623076
1209,BICKENHALL STREET,2.208500e+06,51.521197,-0.158934
1247,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468


In [28]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [29]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [1]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'ULJ50MWMNRKFLLWDI5CV05REIVRUVZFJMPO22NMFQFIRCD3S' # Foursquare ID
CLIENT_SECRET = 'XDODSLUAVDPJQLJ4WA11BZONPS3Q5CJEDWKEVDM3AKRBKLNS' # Foursquare Secret
VERSION = '20190606' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ULJ50MWMNRKFLLWDI5CV05REIVRUVZFJMPO22NMFQFIRCD3S
CLIENT_SECRET:XDODSLUAVDPJQLJ4WA11BZONPS3Q5CJEDWKEVDM3AKRBKLNS


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

### 4. Modeling
After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
AYLESTONE AVENUE
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL ROAD
HORTENSIA ROAD
HOXTON SQUARE
HUN

In [33]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
3,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
4,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
5,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
6,ALBION SQUARE,-41.273758,173.289393,Burger Culture,-41.274750,173.284030,Burger Joint
7,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
8,ALBION SQUARE,-41.273758,173.289393,Fresh Choice,-41.272194,173.287218,Supermarket
9,ALBION SQUARE,-41.273758,173.289393,Mango,-41.274460,173.285345,Indian Restaurant


In [34]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,26,26,26,26,26,26
ANHALT ROAD,15,15,15,15,15,15
ANSDELL TERRACE,53,53,53,53,53,53
APPLEGARTH ROAD,4,4,4,4,4,4
AYLESTONE AVENUE,4,4,4,4,4,4
BARONSMEAD ROAD,10,10,10,10,10,10
BEAUCLERC ROAD,31,31,31,31,31,31
BICKENHALL STREET,100,100,100,100,100,100
BIRCHLANDS AVENUE,9,9,9,9,9,9


In [35]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 343 uniques categories.


In [36]:
location_venues.shape

(5746, 7)

In [37]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ALBION SQUARE,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.038462,...,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,ANHALT ROAD,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,ANSDELL TERRACE,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.018868,...,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.018868,0.000000
3,APPLEGARTH ROAD,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,AYLESTONE AVENUE,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000
5,BARONSMEAD ROAD,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
6,BEAUCLERC ROAD,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7,BICKENHALL STREET,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0000,0.010000,0.0,0.000000,0.000000,0.0,0.010000,0.000000,0.010000,0.010000
8,BIRCHLANDS AVENUE,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
9,BRAMPTON GROVE,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [39]:
london_grouped.shape

(147, 344)

In [40]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.19
1                Bar  0.08
2         Restaurant  0.08
3  Indian Restaurant  0.08
4                Pub  0.08


----ANHALT ROAD----
                  venue  freq
0                   Pub  0.27
1         Grocery Store  0.13
2   Japanese Restaurant  0.07
3                 Plaza  0.07
4  Gym / Fitness Center  0.07


----ANSDELL TERRACE----
                venue  freq
0      Clothing Store  0.08
1  Italian Restaurant  0.08
2           Juice Bar  0.06
3          Restaurant  0.06
4               Hotel  0.06


----APPLEGARTH ROAD----
               venue  freq
0             Casino  0.25
1                Bar  0.25
2          Nightclub  0.25
3                Pub  0.25
4  Accessories Store  0.00


----AYLESTONE AVENUE----
               venue  freq
0           Bus Stop  0.25
1          Wine Shop  0.25
2               Park  0.25
3      Movie Theater  0.25
4  Accessories Store  0.00


----BARONSMEAD ROAD----
             venu

                venue  freq
0               Hotel  0.18
1                 Pub  0.08
2  Chinese Restaurant  0.06
3         Coffee Shop  0.06
4              Garden  0.04


----GERARD ROAD----
             venue  freq
0    Shopping Mall   0.2
1      Supermarket   0.2
2             Lake   0.2
3  Harbor / Marina   0.2
4   Scenic Lookout   0.2


----GERRARD ROAD----
                  venue  freq
0                   Pub  0.09
1           Coffee Shop  0.07
2     French Restaurant  0.05
3  Gym / Fitness Center  0.04
4          Burger Joint  0.04


----GIRDLERS ROAD----
                venue  freq
0                 Pub  0.12
1  Italian Restaurant  0.08
2               Hotel  0.05
3         Pizza Place  0.05
4      Sandwich Place  0.05


----GORDON PLACE----
              venue  freq
0               Bar  0.07
1       Pizza Place  0.06
2    Sandwich Place  0.05
3     Deli / Bodega  0.05
4  Sushi Restaurant  0.04


----GRAFTON SQUARE----
          venue  freq
0           Pub  0.09
1    Restaurant  

                venue  freq
0               Hotel  0.11
1  Italian Restaurant  0.05
2              Bakery  0.05
3         Coffee Shop  0.04
4        Burger Joint  0.04


----PALACE PLACE----
                       venue  freq
0             Sandwich Place  0.17
1                  Gift Shop  0.17
2  Latin American Restaurant  0.17
3                       Food  0.17
4                Pizza Place  0.17


----PANTON STREET----
            venue  freq
0         Theater  0.08
1    Cocktail Bar  0.04
2           Hotel  0.04
3  Ice Cream Shop  0.04
4       Bookstore  0.03


----PARK CRESCENT----
               venue  freq
0               Café  0.23
1                Pub  0.17
2        Coffee Shop  0.10
3        Pizza Place  0.07
4  Fish & Chips Shop  0.07


----PARK LANE----
                 venue  freq
0         Intersection  0.17
1       Sandwich Place  0.17
2      Doctor's Office  0.17
3  American Restaurant  0.17
4        Deli / Bodega  0.17


----PARKE ROAD----
                  venue  freq


                venue  freq
0         IT Services   0.5
1          Playground   0.5
2   Accessories Store   0.0
3           Pet Store   0.0
4  Persian Restaurant   0.0


----THAMES BANK----
              venue  freq
0              Café  0.14
1     Grocery Store  0.07
2  Tapas Restaurant  0.07
3              Park  0.07
4   Harbor / Marina  0.07


----THE HEXAGON----
                  venue  freq
0            Restaurant  0.50
1           Gas Station  0.25
2           Pizza Place  0.25
3     Accessories Store  0.00
4  Pakistani Restaurant  0.00


----TREDEGAR SQUARE----
         venue  freq
0          Pub  0.19
1     Bus Stop  0.11
2  Pizza Place  0.11
3  Coffee Shop  0.11
4          Bar  0.11


----TRINITY STREET----
            venue  freq
0             Pub  0.07
1            Café  0.07
2  Clothing Store  0.06
3     Coffee Shop  0.06
4    Burger Joint  0.05


----UPPER HAMPSTEAD WALK----
                venue  freq
0                Café  0.11
1                 Pub  0.09
2  Italian Resta

In [41]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [43]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [44]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Pub,Bar,Restaurant,Coffee Shop,Indian Restaurant,Park,Fish & Chips Shop,Seafood Restaurant,French Restaurant
1,ANHALT ROAD,Pub,Grocery Store,Japanese Restaurant,English Restaurant,Diner,Bus Stop,Garden,Cocktail Bar,Plaza,Gym / Fitness Center
2,ANSDELL TERRACE,Clothing Store,Italian Restaurant,Juice Bar,Hotel,Restaurant,Café,English Restaurant,Indian Restaurant,French Restaurant,Garden
3,APPLEGARTH ROAD,Nightclub,Bar,Pub,Casino,Yoga Studio,Food Court,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
4,AYLESTONE AVENUE,Wine Shop,Bus Stop,Park,Movie Theater,Yoga Studio,Food Stand,Fish Market,Flea Market,Flower Shop,Food


In [45]:
venues_sorted.shape

(147, 11)

In [46]:
london_grouped.shape

(147, 344)

In [47]:
london_grouped=df

In [48]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 2, 0, 2, 3, 4, 1, 4, 0, 0, 1, 2, 2, 1, 1, 4, 4, 1, 2, 0, 1, 3,
       3, 2, 1, 1, 0, 2, 3, 0, 2, 4, 4, 4, 3, 2, 2, 4, 0, 1, 4, 3, 0, 3,
       0, 3, 0, 0, 2, 4])

In [49]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
195,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
420,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780
698,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874


In [50]:
london_grouped_clustering.shape

(156, 4)

In [51]:
df.shape

(156, 4)

In [52]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [53]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [54]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
195,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Pub,Bar,Restaurant,Coffee Shop,Indian Restaurant,Park,Fish & Chips Shop,Seafood Restaurant,French Restaurant
390,ANHALT ROAD,2.435000e+06,51.480326,-0.166761,2,Pub,Grocery Store,Japanese Restaurant,English Restaurant,Diner,Bus Stop,Garden,Cocktail Bar,Plaza,Gym / Fitness Center
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,0,Clothing Store,Italian Restaurant,Juice Bar,Hotel,Restaurant,Café,English Restaurant,Indian Restaurant,French Restaurant,Garden
420,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780,2,Nightclub,Bar,Pub,Casino,Yoga Studio,Food Court,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
698,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874,3,Wine Shop,Bus Stop,Park,Movie Theater,Yoga Studio,Food Stand,Fish Market,Flea Market,Flower Shop,Food
851,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,4,Nature Preserve,Indie Movie Theater,Park,Coffee Shop,Movie Theater,Breakfast Spot,Restaurant,Pizza Place,Thai Restaurant,Pub
975,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033,1,Pub,Coffee Shop,Hotel,Chinese Restaurant,Grocery Store,Thai Restaurant,Bakery,Falafel Restaurant,Cocktail Bar,Fast Food Restaurant
1096,BELVEDERE DRIVE,2.340000e+06,38.201316,-84.623076,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1209,BICKENHALL STREET,2.208500e+06,51.521197,-0.158934,0,Hotel,Café,Chinese Restaurant,Coffee Shop,Indian Restaurant,Restaurant,Pub,Pizza Place,Gastropub,Sandwich Place
1247,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,0,Pub,Train Station,Coffee Shop,Chinese Restaurant,Brewery,Lake,Bakery,French Restaurant,Yoga Studio,Fish Market


In [55]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Clothing Store,Italian Restaurant,Juice Bar,Hotel,Restaurant,Café,English Restaurant,Indian Restaurant,French Restaurant,Garden
1209,2208500.0,Hotel,Café,Chinese Restaurant,Coffee Shop,Indian Restaurant,Restaurant,Pub,Pizza Place,Gastropub,Sandwich Place
1247,2217000.0,Pub,Train Station,Coffee Shop,Chinese Restaurant,Brewery,Lake,Bakery,French Restaurant,Yoga Studio,Fish Market
2216,2200000.0,Nightlife Spot,Playground,Soccer Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop
2627,2250000.0,Gastropub,Construction & Landscaping,Farmers Market,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop


In [57]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
195,2450000.0,Café,Pub,Bar,Restaurant,Coffee Shop,Indian Restaurant,Park,Fish & Chips Shop,Seafood Restaurant,French Restaurant
975,2480000.0,Pub,Coffee Shop,Hotel,Chinese Restaurant,Grocery Store,Thai Restaurant,Bakery,Falafel Restaurant,Cocktail Bar,Fast Food Restaurant
1546,2456875.0,Middle Eastern Restaurant,Lake,Men's Store,Yoga Studio,Food Stand,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
1906,2445000.0,Café,Diner,Athletics & Sports,Gas Station,Gastropub,Brewery,Stadium,Grocery Store,Fish & Chips Shop,Coffee Shop
1972,2492500.0,Supermarket,English Restaurant,Hardware Store,Rental Car Location,Pub,American Restaurant,Park,Café,Gym,Food


In [58]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2435000.0,Pub,Grocery Store,Japanese Restaurant,English Restaurant,Diner,Bus Stop,Garden,Cocktail Bar,Plaza,Gym / Fitness Center
420,2400000.0,Nightclub,Bar,Pub,Casino,Yoga Studio,Food Court,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
1625,2397132.0,Sporting Goods Shop,Grocery Store,Italian Restaurant,Other Great Outdoors,Coffee Shop,Gym / Fitness Center,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
1790,2400000.0,Middle Eastern Restaurant,Art Gallery,Yoga Studio,Food Truck,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court
2149,2425000.0,Coffee Shop,Hotel,Italian Restaurant,Pub,Bar,Cocktail Bar,Burger Joint,Café,Clothing Store,History Museum


In [59]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
698,2.286667e+06,Wine Shop,Bus Stop,Park,Movie Theater,Yoga Studio,Food Stand,Fish Market,Flea Market,Flower Shop,Food
2233,2.300000e+06,Campground,Yoga Studio,Food Truck,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court
2396,2.286679e+06,Café,Bar,Italian Restaurant,Pizza Place,Pub,Coffee Shop,Vegetarian / Vegan Restaurant,Spa,Modern European Restaurant,Gourmet Shop
2675,2.287500e+06,Pub,Gym / Fitness Center,Gift Shop,Brewery,Art Museum,Yoga Studio,Fish Market,Flea Market,Flower Shop,Food
3363,2.298000e+06,Hotel,Yoga Studio,Farmers Market,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop


In [60]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
851,2375000.0,Nature Preserve,Indie Movie Theater,Park,Coffee Shop,Movie Theater,Breakfast Spot,Restaurant,Pizza Place,Thai Restaurant,Pub
1096,2340000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2059,2375000.0,Pub,Yoga Studio,Park,Hotel,Ice Cream Shop,Indian Restaurant,Coffee Shop,Grocery Store,Bakery,Health & Beauty Service
2120,2379652.7,Pub,Coffee Shop,Bakery,Yoga Studio,Hostel,Indian Restaurant,Grocery Store,Ice Cream Shop,Hotel,Pedestrian Plaza
2931,2367500.0,Hotel,Pub,Garden,Café,Coffee Shop,Italian Restaurant,Bar,Mediterranean Restaurant,Indian Restaurant,Chinese Restaurant


### Results and Discussion section
First of all, even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs.

We may discuss our results under two main perspectives.

First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer.

### Conclusion
To sum up, according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.